In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from sklearn.model_selection import GroupShuffleSplit 

import sys, os
from pathlib import Path

sys.path.insert(0, os.path.join(Path('.').resolve().parent.parent))

from fusionlib.model import predict # Функция, позволяет получить предсказание нейронки.
from fusionlib.check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита

In [2]:
df = pd.read_csv('/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/transactions_finetune.csv')
df.user_id.nunique()

7080

In [10]:
targets = pd.read_csv('/Users/xxx/Documents/Programming/vtb-data-fusion-2023/data/target_finetune.csv')


In [12]:
targets.target.value_counts()

0    6818
1     262
Name: target, dtype: int64

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
# cv = StratifiedGroupKFold(n_splits=3, shuffle=True)
# train_idxs, test_idxs = next(cv.split(df, targets, groups=df['user_id']))

splitter = StratifiedShuffleSplit(test_size=.3, n_splits=1, random_state = 7)
split = splitter.split(targets, targets.target)
train_inds, test_inds = next(split)

test_targets = targets.iloc[test_inds]
test_user_id_set = set(test_targets.user_id.unique())
test = df[df.user_id.apply(lambda id: id in test_user_id_set)]

In [20]:
assert train.user_id.nunique() + test.user_id.nunique() == df.user_id.nunique()

In [29]:
test.to_csv('/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/data/validation_clear_transactions.csv', index_label=False)

# Создание атакованных валидационных данных

In [30]:
df = test

In [31]:
validation_path = '/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/data/validation_clear_transactions.csv'
transactions_path = validation_path
bins_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/models/nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/models/nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/misc/quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)
BUDGET = 10 # разрешенное количество изменений транзакций для каждого пользователя

In [32]:
%%time
# у нас нет разметки для тех транзакций, которые мы атакуем - но у нас есть модель.
# Давайте посчитаем вероятность того, что пользователь принадлежит к классу 1
result = predict(validation_path, bins_path, model_path, random_seed=20230206)
result.head()

Global seed set to 20230206


CPU times: user 31.9 s, sys: 6.57 s, total: 38.5 s
Wall time: 23.1 s


,user_id,target
0,626,0.187110
1,925,0.003806
2,939,0.006336
3,1158,0.060767
4,1259,0.040798


In [33]:
# давайте в качестве порога использовать середину этого диапазона
# все что выше - пусть будет предсказано 1, что ниже - 0
threshold = result.target.max() / 2 
threshold

0.20751450955867767

In [34]:
# Найдем пользователя, для которого нейронка предсказала самое большое значение таргета.
# Это будет наш Герой, Образцовый Положительный Пользователь
hero_user = result.user_id.loc[result.target.argmax()]
hero_user

228615

In [35]:
# Найдем так пользователя с самым низким таргетом
# Это будет наш Неудачник
poor_user = result.user_id.loc[result.target.argmin()]
poor_user

774414

In [36]:
# границы допустимых решений.

with open(quantiles_path, 'r') as f:
    quantiles = json.load(f)

In [37]:
# для каждого кода заданы лимиты положительных и отрифательных значений
# Вот, например, диапазон, в котором должны лежать суммы для ьcc_code 4111
quantiles["positive"]["min"]["4111"], quantiles["positive"]["max"]["4111"]

(8.145073890686035, 45117.69001)

In [38]:
random.seed(20230206)
# Читаем файл с исходными транзакциями

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

In [39]:
%%time
target = predict(transactions_path, bins_path, model_path)

Global seed set to 20230206


CPU times: user 28.8 s, sys: 5.82 s, total: 34.6 s
Wall time: 19.2 s


In [40]:
one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values
one_users = users[one_idx]
zero_users = users[zero_idx]

In [41]:
output_path = "/Users/xxx/Documents/Programming/vtb-data-fusion-2023/vtb-data-fusion-2023-defence/data/validation_fraud_transactions.csv"

In [42]:
for user in tqdm(users):
    if user in one_users:
        copy_from = poor_user # похожим на Героя скопируем 10 последних транзакций Неудачника
    else:
        copy_from = hero_user # А похожим на Неудачника наоборот

    idx_to = df_transactions.index[df_transactions.user_id == user][-BUDGET:]
    idx_from = df_transactions.index[df_transactions.user_id == copy_from][-BUDGET:]
    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)
    df_transactions.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values
df_transactions.to_csv(output_path, index=False)


100%|██████████| 2124/2124 [00:30<00:00, 69.74it/s]


In [43]:
check_budget(transactions_path, output_path, quantiles_path) # Не забываем проверять бюджет перед самбитом!

100%|██████████| 637200/637200 [00:54<00:00, 11726.92it/s]


True